# Router LLM Model Agent - 커피 키오스크 주문 시스템

## 1. 환경 설정 및 라이브러리 임포트

In [78]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

True

In [79]:
# 필요한 라이브러리 설치
%pip install -q pydantic

Note: you may need to restart the kernel to use updated packages.


In [80]:
import os
import json
import time
from typing import List, Dict, Optional, Literal
from datetime import datetime
from openai import OpenAI
from pydantic import BaseModel, Field, validator

# OpenAI 클라이언트 초기화
client = OpenAI()

## 2. Pydantic 모델 정의

In [81]:
class QueryComplexity(BaseModel):
    """사용자 질문의 복잡도를 분석한 결과를 표현하는 모델"""
    complexity_level: Literal["simple", "moderate", "complex"] = Field(
        description="질문의 복잡도: simple(단순), moderate(보통), complex(복잡)"
    )
    reasoning_depth: int = Field(
        ge=1, le=5,
        description="필요한 추론 깊이 (1: 단순 조회, 5: 다단계 추론)"
    )
    response_time_priority: Literal["fast", "balanced", "quality"] = Field(
        description="응답 시간 우선순위"
    )
    requires_creativity: bool = Field(
        description="창의적 답변이 필요한지 여부"
    )
    analysis_reason: str = Field(
        description="복잡도 분석 근거"
    )


class ModelSpecification(BaseModel):
    """LLM 모델의 사양과 특성을 표현하는 모델"""
    model_name: str = Field(description="모델 이름")
    model_tier: Literal["nano", "mini", "standard"] = Field(
        description="모델 계층"
    )
    max_completion_tokens: int = Field(description="최대 토큰 수")
    cost_per_1k_tokens: float = Field(description="1K 토큰당 비용 (USD)")
    avg_latency_ms: int = Field(description="평균 응답 지연시간 (밀리초)")
    suitable_for: List[str] = Field(description="적합한 태스크 유형")


class ModelSelection(BaseModel):
    """모델 선택 결과를 표현하는 모델"""
    selected_model: ModelSpecification = Field(description="선택된 모델")
    confidence: float = Field(
        ge=0.0, le=1.0,
        description="모델 선택에 대한 신뢰도"
    )
    selection_reason: str = Field(description="모델 선택 이유")
    alternative_models: List[str] = Field(
        default_factory=list,
        description="대안 모델 목록"
    )


class ConversationMessage(BaseModel):
    """대화 메시지를 표현하는 모델"""
    role: Literal["user", "assistant", "system"] = Field(description="메시지 역할")
    content: str = Field(description="메시지 내용")
    timestamp: datetime = Field(default_factory=datetime.now, description="메시지 생성 시간")
    model_used: Optional[str] = Field(default=None, description="사용된 모델")
    metadata: Dict = Field(default_factory=dict, description="추가 메타데이터")


class ResponseQuality(BaseModel):
    """생성된 응답의 품질을 표현하는 모델"""
    accuracy: float = Field(ge=0.0, le=1.0, description="정확도")
    completeness: float = Field(ge=0.0, le=1.0, description="완전성")
    clarity: float = Field(ge=0.0, le=1.0, description="명확성")
    relevance: float = Field(ge=0.0, le=1.0, description="관련성")
    overall_score: float = Field(ge=0.0, le=1.0, description="종합 점수")
    
    @validator('overall_score', always=True)
    def calculate_overall_score(cls, v, values):
        """개별 점수들의 평균으로 종합 점수를 계산한다"""
        if all(k in values for k in ['accuracy', 'completeness', 'clarity', 'relevance']):
            return (values['accuracy'] + values['completeness'] + 
                   values['clarity'] + values['relevance']) / 4
        return v


class AgentResponse(BaseModel):
    """에이전트의 최종 응답을 표현하는 모델"""
    answer: str = Field(description="사용자에게 제공되는 답변")
    model_used: str = Field(description="사용된 모델")
    complexity_analysis: QueryComplexity = Field(description="질문 복잡도 분석")
    response_time_ms: int = Field(description="응답 생성 시간 (밀리초)")
    quality_score: ResponseQuality = Field(description="응답 품질 점수")
    was_escalated: bool = Field(description="상위 모델로 에스컬레이션 되었는지 여부")
    total_cost_usd: float = Field(description="총 비용 (USD)")

/var/folders/v9/46y9d8bn1lxgjt7g439hsf8c0000gn/T/ipykernel_94615/1450928007.py:64: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('overall_score', always=True)


## 3. 모델 레지스트리 구축

In [82]:
# 모델 사양 정의 (실제 운영 환경의 값을 시뮬레이션)
MODEL_REGISTRY = {
    "gpt-5-nano": ModelSpecification(
        model_name="gpt-5-nano",  # nano는 mini의 낮은 temperature로 시뮬레이션
        model_tier="nano",
        max_completion_tokens=500,
        cost_per_1k_tokens=0.00015,
        avg_latency_ms=200,
        suitable_for=["인사", "단순 확인", "메뉴 조회", "가격 문의"]
    ),
    "gpt-5-mini": ModelSpecification(
        model_name="gpt-5-mini",
        model_tier="mini",
        max_completion_tokens=2000,
        cost_per_1k_tokens=0.00030,
        avg_latency_ms=500,
        suitable_for=["일반 주문", "메뉴 추천", "옵션 설명", "간단한 조합"]
    ),
    "gpt-4o-mini": ModelSpecification(
        model_name="gpt-4o-mini",  # 실제로는 gpt-4o를 사용하지만 여기서는 mini로 시뮬레이션
        model_tier="standard",
        max_completion_tokens=4000,
        cost_per_1k_tokens=0.00500,
        avg_latency_ms=1500,
        suitable_for=["복잡한 조합", "맞춤형 추천", "다단계 추론", "창의적 제안"]
    )
}

# 모델 계층 순서 (에스컬레이션용)
MODEL_HIERARCHY = ["gpt-5-nano", "gpt-5-mini", "gpt-4o-mini"]

print(f"총 {len(MODEL_REGISTRY)}개의 모델이 등록되었다.")
for model_id, spec in MODEL_REGISTRY.items():
    print(f"  - {model_id} ({spec.model_tier}): {', '.join(spec.suitable_for[:2])}...")

총 3개의 모델이 등록되었다.
  - gpt-5-nano (nano): 인사, 단순 확인...
  - gpt-5-mini (mini): 일반 주문, 메뉴 추천...
  - gpt-4o-mini (standard): 복잡한 조합, 맞춤형 추천...


## 4. Memory 시스템 구현

In [83]:
class ConversationMemory:
    """대화 이력과 모델 사용 이력을 관리하는 메모리 클래스"""
    
    def __init__(self, max_history: int = 10):
        self.messages: List[ConversationMessage] = []
        self.max_history = max_history
        self.model_usage_stats: Dict[str, Dict] = {}
        
        # 각 모델의 사용 통계 초기화
        for model_id in MODEL_REGISTRY.keys():
            self.model_usage_stats[model_id] = {
                "count": 0,
                "total_quality": 0.0,
                "success_count": 0
            }
    
    def add_message(self, role: str, content: str, model_used: str = None, metadata: Dict = None):
        """새로운 메시지를 메모리에 추가한다"""
        message = ConversationMessage(
            role=role,
            content=content,
            model_used=model_used,
            metadata=metadata or {}
        )
        self.messages.append(message)
        
        # 최대 이력 수를 초과하면 오래된 메시지부터 제거한다
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
    
    def update_model_stats(self, model_id: str, quality_score: float, success: bool):
        """모델 사용 통계를 업데이트한다"""
        if model_id in self.model_usage_stats:
            stats = self.model_usage_stats[model_id]
            stats["count"] += 1
            stats["total_quality"] += quality_score
            if success:
                stats["success_count"] += 1
    
    def get_model_performance(self, model_id: str) -> Dict:
        """특정 모델의 성능 통계를 반환한다"""
        if model_id not in self.model_usage_stats:
            return None
        
        stats = self.model_usage_stats[model_id]
        if stats["count"] == 0:
            return {
                "avg_quality": 0.0,
                "success_rate": 0.0,
                "usage_count": 0
            }
        
        return {
            "avg_quality": stats["total_quality"] / stats["count"],
            "success_rate": stats["success_count"] / stats["count"],
            "usage_count": stats["count"]
        }
    
    def get_context(self, include_system: bool = True) -> List[Dict]:
        """OpenAI API 형식으로 대화 이력을 반환한다"""
        context = []
        for msg in self.messages:
            if msg.role == "system" and not include_system:
                continue
            context.append({
                "role": msg.role,
                "content": msg.content
            })
        return context
    
    def get_summary(self) -> str:
        """메모리 상태 요약을 반환한다"""
        total_messages = len(self.messages)
        total_model_uses = sum(stats["count"] for stats in self.model_usage_stats.values())
        return f"메시지: {total_messages}개, 모델 호출: {total_model_uses}회"
    
    def clear(self):
        """모든 이력을 삭제한다"""
        self.messages = []

# 메모리 인스턴스 생성
memory = ConversationMemory(max_history=10)

# 시스템 메시지 추가
memory.add_message(
    "system",
    "당신은 커피 키오스크의 친절한 주문 도우미다. 고객의 질문에 정확하고 친절하게 답변하라."
)

print("대화 메모리가 초기화되었다.")

대화 메모리가 초기화되었다.


## 5. 복잡도 분석 시스템 구현

In [85]:
class ComplexityAnalyzer:
    """질문 복잡도 분석 클래스"""
    
    def __init__(self):
        # 복잡도 분석용 경량 모델 사용
        self.analyzer_model = "gpt-4o-mini"
    
    def analyze(self, query: str, context: List[Dict] = None) -> QueryComplexity:
        """
        사용자 질문의 복잡도를 분석한다
        
        Args:
            query: 사용자 질문
            context: 대화 컨텍스트 (선택적)
        
        Returns:
            QueryComplexity 객체
        """
        analysis_prompt = f"""
다음 커피 키오스크 고객 질문의 복잡도를 분석하라.

질문: "{query}"

다음 기준으로 평가하라:

1. 복잡도 수준 (complexity_level):
   - simple: 단순 인사, 예/아니오 질문, 단일 메뉴 조회
   - moderate: 메뉴 비교, 옵션 설명, 일반적인 추천
   - complex: 다중 조건 추천, 복잡한 조합, 상황별 맞춤 제안

2. 추론 깊이 (reasoning_depth): 1-5
   - 1: 단순 정보 조회
   - 3: 비교 및 추천
   - 5: 다단계 추론 및 창의적 해결

3. 응답 시간 우선순위 (response_time_priority):
   - fast: 즉각 응답 필요
   - balanced: 균형잡힌 속도와 품질
   - quality: 품질 최우선

4. 창의성 필요 (requires_creativity): true/false

5. 분석 근거 (analysis_reason): 한 문장으로 설명

JSON 형식으로만 답변하라:
{{
  "complexity_level": "simple|moderate|complex",
  "reasoning_depth": 1-5,
  "response_time_priority": "fast|balanced|quality",
  "requires_creativity": true|false,
  "analysis_reason": "설명"
}}
"""
        
        messages = [{"role": "user", "content": analysis_prompt}]
        
        response = client.chat.completions.create(
            model=self.analyzer_model,
            messages=messages,
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        
        return QueryComplexity(**result)

# 복잡도 분석기 인스턴스 생성
complexity_analyzer = ComplexityAnalyzer()
print("복잡도 분석 시스템이 초기화되었다.")

복잡도 분석 시스템이 초기화되었다.


## 6. Function Calling & Tool 정의

In [86]:
# 모델 선택을 위한 도구 정의
MODEL_SELECTION_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "select_nano_model",
            "description": "경량 모델을 선택한다. 단순한 질문이나 빠른 응답이 필요한 경우 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reason": {
                        "type": "string",
                        "description": "이 모델을 선택한 이유"
                    }
                },
                "required": ["reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "select_mini_model",
            "description": "표준 모델을 선택한다. 일반적인 주문이나 메뉴 추천에 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reason": {
                        "type": "string",
                        "description": "이 모델을 선택한 이유"
                    }
                },
                "required": ["reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "select_standard_model",
            "description": "고성능 모델을 선택한다. 복잡한 추론이나 창의적 답변이 필요한 경우 사용한다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reason": {
                        "type": "string",
                        "description": "이 모델을 선택한 이유"
                    }
                },
                "required": ["reason"]
            }
        }
    }
]

# 도구 이름과 모델 ID 매핑
TOOL_TO_MODEL_MAP = {
    "select_nano_model": "gpt-5-nano",
    "select_mini_model": "gpt-5-mini",
    "select_standard_model": "gpt-4o-mini"
}

print(f"총 {len(MODEL_SELECTION_TOOLS)}개의 모델 선택 도구가 정의되었다.")

총 3개의 모델 선택 도구가 정의되었다.


## 7. 모델 선택 라우터 구현

In [87]:
class ModelRouter:
    """모델 선택 라우터 클래스"""
    
    def __init__(self, memory: ConversationMemory):
        self.memory = memory
        self.router_model = "gpt-4o-mini"
    
    def route(self, query: str, complexity: QueryComplexity) -> ModelSelection:
        """
        질문과 복잡도 분석 결과를 바탕으로 최적의 모델을 선택한다
        
        Args:
            query: 사용자 질문
            complexity: 복잡도 분석 결과
        
        Returns:
            ModelSelection 객체
        """
        # 복잡도 기반 초기 모델 추천
        routing_prompt = f"""
커피 키오스크 질문에 대해 최적의 LLM 모델을 선택하라.

질문: "{query}"

복잡도 분석:
- 복잡도: {complexity.complexity_level}
- 추론 깊이: {complexity.reasoning_depth}
- 응답 시간 우선순위: {complexity.response_time_priority}
- 창의성 필요: {complexity.requires_creativity}
- 분석 근거: {complexity.analysis_reason}

사용 가능한 모델:
1. gpt-5-nano: 경량 모델 (빠름, 저비용) - {', '.join(MODEL_REGISTRY['gpt-5-nano'].suitable_for)}
2. gpt-5-mini: 표준 모델 (균형) - {', '.join(MODEL_REGISTRY['gpt-5-mini'].suitable_for)}
3. gpt-4o-mini: 고성능 모델 (느림, 고비용) - {', '.join(MODEL_REGISTRY['gpt-4o-mini'].suitable_for)}

과거 모델 성능:
"""
        
        # 모델별 과거 성능 정보 추가
        for model_id in MODEL_HIERARCHY:
            perf = self.memory.get_model_performance(model_id)
            routing_prompt += f"\n- {model_id}: 평균 품질 {perf['avg_quality']:.2f}, 성공률 {perf['success_rate']:.1%}"
        
        routing_prompt += "\n\n가장 적합한 모델을 선택하는 함수를 호출하라."
        
        # Function Calling으로 모델 선택
        response = client.chat.completions.create(
            model=self.router_model,
            messages=[{"role": "user", "content": routing_prompt}],
            tools=MODEL_SELECTION_TOOLS,
            tool_choice="required"
        )
        
        # 도구 호출 결과 파싱
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        # 선택된 모델 ID 가져오기
        selected_model_id = TOOL_TO_MODEL_MAP[function_name]
        selected_spec = MODEL_REGISTRY[selected_model_id]
        
        # 대안 모델 목록 생성
        alternatives = [m for m in MODEL_HIERARCHY if m != selected_model_id]
        
        # 신뢰도 계산 (복잡도와 모델 tier의 적합성 기반)
        confidence = self._calculate_selection_confidence(complexity, selected_spec)
        
        return ModelSelection(
            selected_model=selected_spec,
            confidence=confidence,
            selection_reason=function_args["reason"],
            alternative_models=alternatives
        )
    
    def _calculate_selection_confidence(self, complexity: QueryComplexity, model: ModelSpecification) -> float:
        """
        복잡도와 모델의 적합성을 기반으로 선택 신뢰도를 계산한다
        
        Returns:
            0.0 ~ 1.0 사이의 신뢰도 점수
        """
        # 복잡도와 모델 tier의 매칭도
        tier_match = {
            ("simple", "nano"): 0.95,
            ("simple", "mini"): 0.70,
            ("simple", "standard"): 0.50,
            ("moderate", "nano"): 0.60,
            ("moderate", "mini"): 0.95,
            ("moderate", "standard"): 0.75,
            ("complex", "nano"): 0.30,
            ("complex", "mini"): 0.70,
            ("complex", "standard"): 0.95,
        }
        
        base_confidence = tier_match.get((complexity.complexity_level, model.model_tier), 0.5)
        
        # 창의성 요구와 모델 tier 고려
        if complexity.requires_creativity and model.model_tier in ["mini", "standard"]:
            base_confidence += 0.05
        
        return min(base_confidence, 1.0)

# 모델 라우터 인스턴스 생성
model_router = ModelRouter(memory)
print("모델 라우터가 초기화되었다.")

모델 라우터가 초기화되었다.


## 8. Validation 시스템 구현

In [88]:
class ValidationSystem:
    """모델 선택 및 응답 품질 검증 시스템 클래스"""
    
    def __init__(self, min_confidence: float = 0.6, min_quality: float = 0.7):
        self.min_confidence = min_confidence
        self.min_quality = min_quality
        self.validator_model = "gpt-4o-mini"
    
    def validate_model_selection(self, selection: ModelSelection, complexity: QueryComplexity) -> tuple[bool, str]:
        """
        모델 선택의 적합성을 검증한다
        
        Returns:
            (검증 통과 여부, 검증 메시지)
        """
        # 신뢰도 확인
        if selection.confidence < self.min_confidence:
            return False, f"모델 선택 신뢰도가 낮다 ({selection.confidence:.2f} < {self.min_confidence})"
        
        # 복잡도와 모델의 불일치 확인
        if complexity.complexity_level == "complex" and selection.selected_model.model_tier == "nano":
            return False, "복잡한 질문에 경량 모델이 선택되었다"
        
        if complexity.complexity_level == "simple" and selection.selected_model.model_tier == "standard":
            return False, "단순한 질문에 고성능 모델이 선택되었다 (비효율적)"
        
        return True, "모델 선택이 적합하다"
    
    def validate_response(self, query: str, answer: str, model_used: str) -> ResponseQuality:
        """
        생성된 응답의 품질을 평가한다
        
        Args:
            query: 사용자 질문
            answer: 생성된 답변
            model_used: 사용된 모델
        
        Returns:
            ResponseQuality 객체
        """
        validation_prompt = f"""
커피 키오스크 질문과 답변의 품질을 평가하라.

질문: {query}
답변: {answer}
사용 모델: {model_used}

다음 기준으로 0.0-1.0 점수를 매겨라:
1. accuracy: 답변의 정확성 (사실 관계가 올바른가?)
2. completeness: 답변의 완전성 (질문에 충분히 답했는가?)
3. clarity: 답변의 명확성 (이해하기 쉬운가?)
4. relevance: 답변의 관련성 (질문과 관련있는가?)

JSON 형식으로만 답변하라:
{{
  "accuracy": 0.0-1.0,
  "completeness": 0.0-1.0,
  "clarity": 0.0-1.0,
  "relevance": 0.0-1.0,
  "overall_score": 0.0-1.0
}}
"""
        
        response = client.chat.completions.create(
            model=self.validator_model,
            messages=[{"role": "user", "content": validation_prompt}],
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        return ResponseQuality(**result)
    
    def should_escalate(self, quality: ResponseQuality) -> bool:
        """
        응답 품질이 낮아 상위 모델로 에스컬레이션해야 하는지 판단한다
        
        Returns:
            에스컬레이션 필요 여부
        """
        return quality.overall_score < self.min_quality

# 검증 시스템 인스턴스 생성
validator = ValidationSystem(min_confidence=0.6, min_quality=0.7)
print("검증 시스템이 초기화되었다.")

검증 시스템이 초기화되었다.


## 9. Recovery 메커니즘 구현

In [89]:
class RecoverySystem:
    """복구 및 에스컬레이션 메커니즘 시스템 클래스"""
    
    def __init__(self):
        self.escalation_history = []
        self.max_escalations = 2
    
    def get_next_model(self, current_model_id: str) -> Optional[str]:
        """
        현재 모델보다 상위 모델을 반환한다
        
        Args:
            current_model_id: 현재 모델 ID
        
        Returns:
            다음 모델 ID 또는 None (최상위 모델인 경우)
        """
        current_idx = MODEL_HIERARCHY.index(current_model_id)
        
        if current_idx >= len(MODEL_HIERARCHY) - 1:
            return None
        
        return MODEL_HIERARCHY[current_idx + 1]
    
    def attempt_escalation(
        self,
        query: str,
        current_model_id: str,
        failure_reason: str
    ) -> tuple[Optional[str], str]:
        """
        상위 모델로 에스컬레이션을 시도한다
        
        Args:
            query: 사용자 질문
            current_model_id: 현재 실패한 모델 ID
            failure_reason: 실패 이유
        
        Returns:
            (다음 모델 ID, 메시지)
        """
        # 에스컬레이션 이력 확인
        if len(self.escalation_history) >= self.max_escalations:
            return None, "최대 에스컬레이션 횟수에 도달했다"
        
        # 다음 모델 가져오기
        next_model_id = self.get_next_model(current_model_id)
        
        if next_model_id is None:
            return None, "이미 최상위 모델이다"
        
        # 에스컬레이션 기록
        self.escalation_history.append({
            "from_model": current_model_id,
            "to_model": next_model_id,
            "reason": failure_reason,
            "timestamp": datetime.now()
        })
        
        print(f"Recovery: {current_model_id} → {next_model_id} 에스컬레이션 ({failure_reason})")
        
        return next_model_id, f"{current_model_id}에서 {next_model_id}로 에스컬레이션되었다"
    
    def generate_fallback_response(self, query: str) -> str:
        """
        모든 복구 시도가 실패했을 때 대체 응답을 생성한다
        
        Args:
            query: 원본 질문
        
        Returns:
            대체 답변
        """
        return f"죄송하지만 '{query}'에 대한 적절한 답변을 생성하지 못했다. 직원에게 직접 문의해 주시기 바란다."
    
    def reset(self):
        """에스컬레이션 이력을 초기화한다"""
        self.escalation_history = []
    
    def get_escalation_stats(self) -> Dict:
        """에스컬레이션 통계를 반환한다"""
        if not self.escalation_history:
            return {"count": 0, "most_common_reason": None}
        
        reasons = [e["reason"] for e in self.escalation_history]
        most_common = max(set(reasons), key=reasons.count) if reasons else None
        
        return {
            "count": len(self.escalation_history),
            "most_common_reason": most_common
        }

# 복구 시스템 인스턴스 생성
recovery_system = RecoverySystem()
print("복구 시스템이 초기화되었다.")

복구 시스템이 초기화되었다.


## 10. Feedback 시스템 구현

In [90]:
class FeedbackSystem:
    """피드백 및 학습 시스템 클래스"""
    
    def __init__(self):
        self.feedback_records = []
    
    def record_interaction(
        self,
        query: str,
        complexity: QueryComplexity,
        model_used: str,
        answer: str,
        quality: ResponseQuality,
        was_escalated: bool
    ):
        """
        상호작용 기록을 저장한다
        
        Args:
            query: 사용자 질문
            complexity: 복잡도 분석 결과
            model_used: 사용된 모델
            answer: 생성된 답변
            quality: 품질 평가 결과
            was_escalated: 에스컬레이션 여부
        """
        record = {
            "query": query,
            "complexity_level": complexity.complexity_level,
            "model_used": model_used,
            "quality_score": quality.overall_score,
            "was_escalated": was_escalated,
            "timestamp": datetime.now()
        }
        self.feedback_records.append(record)
    
    def analyze_model_performance(self) -> Dict[str, Dict]:
        """
        각 모델의 성능을 분석한다
        
        Returns:
            모델별 성능 통계
        """
        performance = {}
        
        for model_id in MODEL_HIERARCHY:
            model_records = [r for r in self.feedback_records if r["model_used"] == model_id]
            
            if not model_records:
                performance[model_id] = {
                    "usage_count": 0,
                    "avg_quality": 0.0,
                    "escalation_rate": 0.0
                }
                continue
            
            avg_quality = sum(r["quality_score"] for r in model_records) / len(model_records)
            escalation_rate = sum(r["was_escalated"] for r in model_records) / len(model_records)
            
            performance[model_id] = {
                "usage_count": len(model_records),
                "avg_quality": avg_quality,
                "escalation_rate": escalation_rate
            }
        
        return performance
    
    def get_optimization_suggestions(self) -> List[str]:
        """
        성능 분석을 바탕으로 최적화 제안을 생성한다
        
        Returns:
            최적화 제안 리스트
        """
        if not self.feedback_records:
            return ["데이터가 충분하지 않다."]
        
        suggestions = []
        performance = self.analyze_model_performance()
        
        # 각 모델별 분석
        for model_id, stats in performance.items():
            if stats["usage_count"] == 0:
                continue
            
            # 에스컬레이션 비율이 높은 경우
            if stats["escalation_rate"] > 0.3:
                suggestions.append(
                    f"{model_id}의 에스컬레이션 비율이 {stats['escalation_rate']:.1%}로 높다. "
                    f"이 모델의 사용 기준을 재검토하라."
                )
            
            # 품질이 낮은 경우
            if stats["avg_quality"] < 0.7:
                suggestions.append(
                    f"{model_id}의 평균 품질이 {stats['avg_quality']:.2f}로 낮다. "
                    f"더 높은 tier의 모델 사용을 고려하라."
                )
        
        # 전체 에스컬레이션 비율
        total_escalations = sum(r["was_escalated"] for r in self.feedback_records)
        escalation_rate = total_escalations / len(self.feedback_records)
        
        if escalation_rate > 0.2:
            suggestions.append(
                f"전체 에스컬레이션 비율이 {escalation_rate:.1%}다. "
                f"초기 모델 선택 기준을 상향 조정하는 것을 고려하라."
            )
        
        return suggestions if suggestions else ["현재 모델 선택 전략이 잘 작동하고 있다."]
    
    def get_cost_analysis(self) -> Dict:
        """
        비용 분석을 수행한다
        
        Returns:
            비용 분석 결과
        """
        if not self.feedback_records:
            return {"total_cost": 0.0, "avg_cost_per_query": 0.0}
        
        # 간단한 비용 추정 (실제로는 토큰 수를 계산해야 함)
        total_cost = 0.0
        for record in self.feedback_records:
            model_spec = MODEL_REGISTRY[record["model_used"]]
            # 평균 500 토큰 사용으로 가정
            estimated_cost = model_spec.cost_per_1k_tokens * 0.5
            total_cost += estimated_cost
        
        return {
            "total_cost": total_cost,
            "avg_cost_per_query": total_cost / len(self.feedback_records),
            "query_count": len(self.feedback_records)
        }

# 피드백 시스템 인스턴스 생성
feedback_system = FeedbackSystem()
print("피드백 시스템이 초기화되었다.")

피드백 시스템이 초기화되었다.


## 11. Router LLM Model Agent 통합 구현

In [91]:
class RouterLLMModelAgent:
    """Router LLM Model Agent 메인 클래스"""
    
    def __init__(
        self,
        complexity_analyzer: ComplexityAnalyzer,
        model_router: ModelRouter,
        memory: ConversationMemory,
        validator: ValidationSystem,
        recovery: RecoverySystem,
        feedback: FeedbackSystem
    ):
        self.complexity_analyzer = complexity_analyzer
        self.model_router = model_router
        self.memory = memory
        self.validator = validator
        self.recovery = recovery
        self.feedback = feedback
    
    def generate_response(self, query: str, model_id: str) -> tuple[str, int]:
        """
        선택된 모델로 응답을 생성한다
        
        Args:
            query: 사용자 질문
            model_id: 사용할 모델 ID
        
        Returns:
            (생성된 답변, 응답 시간 ms)
        """
        model_spec = MODEL_REGISTRY[model_id]
        
        # 대화 컨텍스트 가져오기
        messages = self.memory.get_context()
        messages.append({"role": "user", "content": query})
        
        # 응답 생성 시간 측정
        start_time = time.time()
        
        response = client.chat.completions.create(
            model=model_spec.model_name,
            messages=messages,
            max_completion_tokens=model_spec.max_completion_tokens,
        )
        
        end_time = time.time()
        response_time_ms = int((end_time - start_time) * 1000)
        
        answer = response.choices[0].message.content.strip()
        
        return answer, response_time_ms
    
    def process_query(self, user_query: str) -> AgentResponse:
        """
        사용자 질문을 처리하고 최종 응답을 생성한다
        
        Args:
            user_query: 사용자 질문
        
        Returns:
            AgentResponse 객체
        """
        print(f"\n{'='*60}")
        print(f"사용자 질문: {user_query}")
        print(f"{'='*60}")
        
        # Recovery 초기화
        self.recovery.reset()
        
        # 1. 사용자 메시지를 메모리에 추가
        self.memory.add_message("user", user_query)
        
        # 2. 질문 복잡도 분석
        print("\n단계 1: 질문 복잡도 분석")
        complexity = self.complexity_analyzer.analyze(user_query)
        print(f"  - 복잡도: {complexity.complexity_level}")
        print(f"  - 추론 깊이: {complexity.reasoning_depth}")
        print(f"  - 분석 근거: {complexity.analysis_reason}")
        
        # 3. 모델 선택
        print("\n단계 2: 모델 선택")
        selection = self.model_router.route(user_query, complexity)
        print(f"  - 선택된 모델: {selection.selected_model.model_tier} tier")
        print(f"  - 신뢰도: {selection.confidence:.2f}")
        print(f"  - 선택 이유: {selection.selection_reason}")
        
        # 4. 모델 선택 검증
        is_valid, validation_msg = self.validator.validate_model_selection(selection, complexity)
        print(f"\n단계 3: 모델 선택 검증")
        print(f"  - 검증 결과: {validation_msg}")
        
        # 선택 검증 실패 시 대안 모델 사용
        if not is_valid and selection.alternative_models:
            print("  - 대안 모델로 전환")
            alternative_id = selection.alternative_models[0]
            selection.selected_model = MODEL_REGISTRY[alternative_id]
        
        # 현재 사용할 모델 ID
        current_model_id = None
        for model_id, spec in MODEL_REGISTRY.items():
            if spec == selection.selected_model:
                current_model_id = model_id
                break
        
        # 5. 응답 생성 (에스컬레이션 루프)
        answer = None
        quality = None
        response_time_ms = 0
        was_escalated = False
        attempts = 0
        max_attempts = 3
        
        while attempts < max_attempts:
            attempts += 1
            print(f"\n단계 4-{attempts}: 응답 생성 (모델: {current_model_id})")
            
            # 응답 생성
            answer, response_time_ms = self.generate_response(user_query, current_model_id)
            print(f"  - 응답 시간: {response_time_ms}ms")
            print(f"  - 답변: {answer[:100]}..." if len(answer) > 100 else f"  - 답변: {answer}")
            
            # 6. 응답 품질 검증
            quality = self.validator.validate_response(user_query, answer, current_model_id)
            print(f"\n단계 5-{attempts}: 응답 품질 검증")
            print(f"  - 정확도: {quality.accuracy:.2f}")
            print(f"  - 완전성: {quality.completeness:.2f}")
            print(f"  - 명확성: {quality.clarity:.2f}")
            print(f"  - 관련성: {quality.relevance:.2f}")
            print(f"  - 종합 점수: {quality.overall_score:.2f}")
            
            # 7. 에스컬레이션 판단
            should_escalate = self.validator.should_escalate(quality)
            
            if not should_escalate:
                print("  - 품질이 충분하다. 응답 확정.")
                break
            
            # 에스컬레이션 시도
            print(f"  - 품질이 낮다 ({quality.overall_score:.2f}). 에스컬레이션 시도...")
            next_model_id, escalation_msg = self.recovery.attempt_escalation(
                user_query,
                current_model_id,
                f"품질 점수 {quality.overall_score:.2f}"
            )
            
            if next_model_id is None:
                print(f"  - {escalation_msg}. 현재 응답 사용.")
                break
            
            current_model_id = next_model_id
            was_escalated = True
        
        # 최종 응답이 없으면 대체 응답 생성
        if answer is None:
            answer = self.recovery.generate_fallback_response(user_query)
            quality = ResponseQuality(
                accuracy=0.0,
                completeness=0.0,
                clarity=0.0,
                relevance=0.0,
                overall_score=0.0
            )
        
        # 8. 메모리에 답변 추가
        self.memory.add_message(
            "assistant",
            answer,
            model_used=current_model_id,
            metadata={
                "quality_score": quality.overall_score,
                "was_escalated": was_escalated
            }
        )
        
        # 9. 모델 사용 통계 업데이트
        self.memory.update_model_stats(
            current_model_id,
            quality.overall_score,
            quality.overall_score >= 0.7
        )
        
        # 10. 피드백 기록
        self.feedback.record_interaction(
            user_query,
            complexity,
            current_model_id,
            answer,
            quality,
            was_escalated
        )
        
        # 11. 비용 계산
        model_spec = MODEL_REGISTRY[current_model_id]
        estimated_cost = model_spec.cost_per_1k_tokens * 0.5  # 평균 500 토큰 가정
        
        # 12. 최종 응답 생성
        return AgentResponse(
            answer=answer,
            model_used=current_model_id,
            complexity_analysis=complexity,
            response_time_ms=response_time_ms,
            quality_score=quality,
            was_escalated=was_escalated,
            total_cost_usd=estimated_cost
        )

# Router LLM Model Agent 인스턴스 생성
agent = RouterLLMModelAgent(
    complexity_analyzer=complexity_analyzer,
    model_router=model_router,
    memory=memory,
    validator=validator,
    recovery=recovery_system,
    feedback=feedback_system
)

print("\nRouter LLM Model Agent가 초기화되었다.")


Router LLM Model Agent가 초기화되었다.


## 12. 에이전트 테스트

In [92]:
# 테스트 질문 리스트 (복잡도 다양화)
test_queries = [
    "안녕하세요",  # Simple
    "아메리카노 가격은 얼마인가요?",  # Simple
    "카페라떼와 카푸치노 중 뭐가 더 부드러운가요?",  # Moderate
    "피곤한데 잠이 안 오면서도 집중력을 높일 수 있는 음료를 추천해주세요. 우유는 먹을 수 있지만 너무 달지 않았으면 좋겠어요.",  # Complex
]

# 각 질문에 대해 에이전트 실행
for query in test_queries:
    response = agent.process_query(query)
    print(f"\n{'='*60}")
    print(f"최종 응답 요약")
    print(f"{'='*60}")
    print(f"답변: {response.answer}")
    print(f"사용 모델: {response.model_used}")
    print(f"복잡도: {response.complexity_analysis.complexity_level}")
    print(f"응답 시간: {response.response_time_ms}ms")
    print(f"품질 점수: {response.quality_score.overall_score:.2f}")
    print(f"에스컬레이션: {'예' if response.was_escalated else '아니오'}")
    print(f"예상 비용: ${response.total_cost_usd:.6f}")
    print()


사용자 질문: 안녕하세요

단계 1: 질문 복잡도 분석
  - 복잡도: simple
  - 추론 깊이: 1
  - 분석 근거: 단순 인사로, 별다른 정보 조회나 복잡한 요구가 없기 때문에 매우 간단하다.

단계 2: 모델 선택
  - 선택된 모델: nano tier
  - 신뢰도: 0.95
  - 선택 이유: 최소한의 복잡도와 빠른 응답을 제공하기 위해 경량 모델을 선택했다.

단계 3: 모델 선택 검증
  - 검증 결과: 모델 선택이 적합하다

단계 4-1: 응답 생성 (모델: gpt-5-nano)
  - 응답 시간: 6057ms
  - 답변: 

단계 5-1: 응답 품질 검증
  - 정확도: 0.00
  - 완전성: 0.00
  - 명확성: 0.00
  - 관련성: 0.00
  - 종합 점수: 0.00
  - 품질이 낮다 (0.00). 에스컬레이션 시도...
Recovery: gpt-5-nano → gpt-5-mini 에스컬레이션 (품질 점수 0.00)

단계 4-2: 응답 생성 (모델: gpt-5-mini)
  - 응답 시간: 11439ms
  - 답변: 안녕하세요! 반갑습니다. 주문 도와드릴게요. 무엇을 드릴까요?

추천 메뉴 몇 가지 말씀드릴게요.
- 아메리카노: 깔끔하고 부담 없음  
- 카페라떼: 부드러운 우유 풍미  
- ...

단계 5-2: 응답 품질 검증
  - 정확도: 1.00
  - 완전성: 1.00
  - 명확성: 1.00
  - 관련성: 1.00
  - 종합 점수: 1.00
  - 품질이 충분하다. 응답 확정.

최종 응답 요약
답변: 안녕하세요! 반갑습니다. 주문 도와드릴게요. 무엇을 드릴까요?

추천 메뉴 몇 가지 말씀드릴게요.
- 아메리카노: 깔끔하고 부담 없음  
- 카페라떼: 부드러운 우유 풍미  
- 카라멜 마끼아또: 달콤한 토핑 원하실 때  
- 시즌 음료: 계절 한정 맛(원하시면 지금 메뉴 알려드릴게요)

옵션: 사이즈(스몰/미디움/라지), HOT/ICED, 샷 추가, 시럽/당도 조절, 우유 변경(오트/저지방/두유) 

## 13. 대화형 인터페이스

In [94]:
def chat_interface():
    """대화형 인터페이스 함수"""
    print("\n" + "="*60)
    print("커피 키오스크 Router LLM Model Agent")
    print("종료하려면 'quit' 또는 'exit'를 입력하세요.")
    print("="*60 + "\n")
    
    while True:
        user_input = input("\n고객: ").strip()
        
        if user_input.lower() in ['quit', 'exit', '종료']:
            print("\n대화를 종료한다.")
            break
        
        if not user_input:
            continue
        
        response = agent.process_query(user_input)
        print(f"\n에이전트: {response.answer}")
        print(f"[{response.model_used} | 품질: {response.quality_score.overall_score:.2f} | {response.response_time_ms}ms]")

# 대화 시작 (주석을 제거하여 실행)
chat_interface()


커피 키오스크 Router LLM Model Agent
종료하려면 'quit' 또는 'exit'를 입력하세요.




고객:  뜨거운 아메리카노를 에스프레소 2샷 넣어서 만들어주세요



사용자 질문: 뜨거운 아메리카노를 에스프레소 2샷 넣어서 만들어주세요

단계 1: 질문 복잡도 분석
  - 복잡도: simple
  - 추론 깊이: 1
  - 분석 근거: 고객이 요청하는 음료의 종류와 조합이 명확하여 단순한 주문으로 분류된다.

단계 2: 모델 선택
  - 선택된 모델: nano tier
  - 신뢰도: 0.95
  - 선택 이유: 주문 요청이 간단하고 명확하여 빠르고 저비용의 경량 모델이 최적이다.

단계 3: 모델 선택 검증
  - 검증 결과: 모델 선택이 적합하다

단계 4-1: 응답 생성 (모델: gpt-5-nano)
  - 응답 시간: 6048ms
  - 답변: 

단계 5-1: 응답 품질 검증
  - 정확도: 1.00
  - 완전성: 1.00
  - 명확성: 1.00
  - 관련성: 1.00
  - 종합 점수: 1.00
  - 품질이 충분하다. 응답 확정.

에이전트: 
[gpt-5-nano | 품질: 1.00 | 6048ms]



고객:  카페라떼에 우유 대신에 두유를 넣으면서 라지 사이즈 3잔으로 주문할게요



사용자 질문: 카페라떼에 우유 대신에 두유를 넣으면서 라지 사이즈 3잔으로 주문할게요

단계 1: 질문 복잡도 분석
  - 복잡도: complex
  - 추론 깊이: 4
  - 분석 근거: 고객의 주문은 특정 옵션(우유 대신 두유)과 수량(3잔)을 포함한 복합적인 요청이므로 복잡도가 높다.

단계 2: 모델 선택
  - 선택된 모델: standard tier
  - 신뢰도: 0.95
  - 선택 이유: 주문의 복잡성이 높고, 특정 옵션 및 수량을 포함한 맞춤형 주문으로, 고성능 모델이 필요하다.

단계 3: 모델 선택 검증
  - 검증 결과: 모델 선택이 적합하다

단계 4-1: 응답 생성 (모델: gpt-4o-mini)
  - 응답 시간: 2396ms
  - 답변: 주문 내용 확인했습니다! 

- **카페라떼**, 라지 사이즈 3잔 
- **두유로 대체**

주문이 잘 준비될 거예요. 혹시 추가로 시럽이나 다른 옵션이 필요하신가요? 아니면 결...

단계 5-1: 응답 품질 검증
  - 정확도: 1.00
  - 완전성: 0.90
  - 명확성: 1.00
  - 관련성: 1.00
  - 종합 점수: 0.97
  - 품질이 충분하다. 응답 확정.

에이전트: 주문 내용 확인했습니다! 

- **카페라떼**, 라지 사이즈 3잔 
- **두유로 대체**

주문이 잘 준비될 거예요. 혹시 추가로 시럽이나 다른 옵션이 필요하신가요? 아니면 결제 방법도 알려주시면 좋을 것 같아요!
[gpt-4o-mini | 품질: 0.97 | 2396ms]



고객:  quit



대화를 종료한다.


## 14. 성능 분석 및 리포트

In [95]:
print("\n" + "="*60)
print("성능 분석 리포트")
print("="*60)

# 모델별 성능 분석
print("\n1. 모델별 성능")
performance = feedback_system.analyze_model_performance()
for model_id, stats in performance.items():
    print(f"\n  {model_id}:")
    print(f"    - 사용 횟수: {stats['usage_count']}회")
    print(f"    - 평균 품질: {stats['avg_quality']:.2f}")
    print(f"    - 에스컬레이션 비율: {stats['escalation_rate']:.1%}")

# 비용 분석
print("\n2. 비용 분석")
cost_analysis = feedback_system.get_cost_analysis()
print(f"  - 총 비용: ${cost_analysis['total_cost']:.6f}")
print(f"  - 쿼리당 평균 비용: ${cost_analysis['avg_cost_per_query']:.6f}")
print(f"  - 처리한 쿼리 수: {cost_analysis['query_count']}개")

# 최적화 제안
print("\n3. 최적화 제안")
suggestions = feedback_system.get_optimization_suggestions()
for i, suggestion in enumerate(suggestions, 1):
    print(f"  {i}. {suggestion}")

# 에스컬레이션 통계
print("\n4. 에스컬레이션 통계")
escalation_stats = recovery_system.get_escalation_stats()
print(f"  - 총 에스컬레이션 횟수: {escalation_stats['count']}회")
if escalation_stats['most_common_reason']:
    print(f"  - 가장 흔한 이유: {escalation_stats['most_common_reason']}")

# 메모리 상태
print("\n5. 메모리 상태")
print(f"  - {memory.get_summary()}")

print("\n" + "="*60)
print("튜토리얼 2: Router LLM Model Agent 완료")
print("="*60)


성능 분석 리포트

1. 모델별 성능

  gpt-5-nano:
    - 사용 횟수: 1회
    - 평균 품질: 1.00
    - 에스컬레이션 비율: 0.0%

  gpt-5-mini:
    - 사용 횟수: 3회
    - 평균 품질: 1.00
    - 에스컬레이션 비율: 66.7%

  gpt-4o-mini:
    - 사용 횟수: 2회
    - 평균 품질: 0.99
    - 에스컬레이션 비율: 0.0%

2. 비용 분석
  - 총 비용: $0.005525
  - 쿼리당 평균 비용: $0.000921
  - 처리한 쿼리 수: 6개

3. 최적화 제안
  1. gpt-5-mini의 에스컬레이션 비율이 66.7%로 높다. 이 모델의 사용 기준을 재검토하라.
  2. 전체 에스컬레이션 비율이 33.3%다. 초기 모델 선택 기준을 상향 조정하는 것을 고려하라.

4. 에스컬레이션 통계
  - 총 에스컬레이션 횟수: 0회

5. 메모리 상태
  - 메시지: 10개, 모델 호출: 6회

튜토리얼 2: Router LLM Model Agent 완료
